In [1]:
import stf

In [2]:
from pathlib import Path

verbose= False

config_path = './211230/front_config.json'
reference_face = './211230/hunet.png'

work_root_path = './stf_api_temp_root'
device = 'cuda:1'
front_template_video_path = './211230/demo_src/Est Soft 2_1.0-80.mov' # 보라색 자켓, 고개 끄덕 템플릿 (고개 간혹 끄덕임 주문했으나 거의 정자세)
front_checkpoint_path = f'./211230/front_1213_1140_105.pth'
side_template_video_path = './211230/demo_src/Est Soft 2_2.0-80.mov' # 보라색 자켓, 고개 끄덕 템플릿 (고개 간혹 끄덕임 주문했으나 거의 정자세)
side_checkpoint_path = f'./211230/side_1214_1245_039.pth.filepart'


In [3]:
import os
assert(os.path.exists(config_path))
assert(os.path.exists(reference_face))
assert(os.path.exists(front_template_video_path))
assert(os.path.exists(front_checkpoint_path))
assert(os.path.exists(side_template_video_path))
assert(os.path.exists(side_checkpoint_path))

In [4]:
%%time 
def process_callback(per: int) -> None:
    print(per)
    
# template video 전처리 : 처음 실행되었을 때만 실제로 전처리가 발생하고, 이 후 여러번 실행시키면 전처리 되어있는지 확인과정만 일어난다.
stf.preprocess_template(config_path, front_template_video_path,
                        reference_face=reference_face,
                        work_root_path=work_root_path,
                        call_back=process_callback,
                        device=device,
                        verbose=verbose)
stf.preprocess_template(config_path, side_template_video_path,
                        reference_face=reference_face,
                        work_root_path=work_root_path,
                        call_back=process_callback,
                        device=device,
                        verbose=verbose)

load MTCNN  success ^ ^
load g_recognizer  success ^ ^
CPU times: user 3.64 s, sys: 1.46 s, total: 5.1 s
Wall time: 6.68 s


In [5]:
%%time 
front_model = stf.create_model(config_path, front_checkpoint_path, work_root_path, device=device, verbose=verbose)

CPU times: user 1.68 s, sys: 1.39 s, total: 3.07 s
Wall time: 4.26 s


In [6]:
%%time 
front_template = stf.template_video(front_model, front_template_video_path, callback=None, verbose=verbose)

CPU times: user 9.01 s, sys: 31.7 s, total: 40.7 s
Wall time: 1min 30s


In [7]:
%%time 
from pathlib import Path

for i in range(1):
    print('###################### ', i, ' ##############################')
    wav_path = '211230/hunet_only_voice_test_50000.wav'
    wav_std=True
    wav_std_ref_wav=None
    name_prefix=''
    slow_write = True # False
    video_start_offset_frame = 0 # 0이상 숫자: video 해당 frame부터 합성 / None: 맨 마지막에 맞춰서 합성
    
    out_path = 'temp_result/' + f'{name_prefix}_{Path(front_template_video_path).stem}_{Path(wav_path).stem}_mask-{front_template.model.args.mask_ver}_{i}.mp4'
    front_template.gen(wav_path, wav_std, wav_std_ref_wav,
                       video_start_offset_frame, out_path=out_path,
                       slow_write=slow_write)

######################  0  ##############################
mel_step_size:108, mel_ps:80
CPU times: user 1min 18s, sys: 1min 15s, total: 2min 34s
Wall time: 8min 12s


In [8]:
import IPython.display as ipd
ipd.Video(out_path, width=1280)

In [9]:
# 템플릿 비디오 삭제
print('del template')
del front_template

# 모델 삭제
print('del model')
del front_model


del template
del template exception
del model
del model exception
